# Analiza Kontekstu dla Pytań bez Poprawnych Pasaży

Ten notebook analizuje pytania, dla których retriever nie znalazł poprawnego kontekstu (hasCorrectPassages = FALSE).

Dla konfiguracji:
- RETRIEVER: basic_index-ipipan-polqa-500
- GENERATOR: PLLuM-8B-instruct-q4
- DATASET: polqa
- TOP_N: 1

Sprawdzimy:
1. Które pytania nie miały poprawnego kontekstu
2. Jaki kontekst został zwrócony zamiast poprawnego
3. Jaką odpowiedź wygenerował model na podstawie błędnego kontekstu
4. Czy model zahallucynował czy użył błędnego kontekstu

In [1]:
import sys
sys.path.append("../")

In [2]:
from elasticsearch import Elasticsearch
from cache.cache import Cache
from common.names import DATASET_SEED
from repository.es_repository import ESRepository
from retrievers.es_retriever import ESRetriever
from dataset.polqa_dataset_getter import PolqaDatasetGetter

es_client = Elasticsearch(hosts=["http://localhost:9200"])
cache = Cache()

## Załaduj Dataset i Retriever

In [3]:
# Załaduj te same 100 pytań co w ewaluacji
polqa_dataset_getter = PolqaDatasetGetter()
polqa_dataset = polqa_dataset_getter.get_random_n_test(500, DATASET_SEED)[:100]

print(f"Załadowano {len(polqa_dataset)} pytań z PolQA")

Załadowano 100 pytań z PolQA


In [4]:
# Stwórz retriever
dataset_key = "ipipan-polqa-500"
es_index = "basic_index"

es_repository = ESRepository(es_client, es_index, cache)
retriever = ESRetriever(es_repository, dataset_key)

print(f"Retriever: {es_index} z dataset_key: {dataset_key}")

Retriever: basic_index z dataset_key: ipipan-polqa-500


## Analiza Pytań bez Poprawnego Kontekstu

In [5]:
# Dla każdego pytania sprawdź czy znaleziono poprawny kontekst
missing_context_cases = []

for i, entry in enumerate(polqa_dataset):
    question = entry.question
    correct_passage_id = entry.passage_id
    correct_answers = entry.answers
    
    # Pobierz wyniki z retrievera (top 1)
    retriever_result = retriever.get_relevant_passages(question)
    passages = [passage for (passage, _) in retriever_result.passages]
    top_1_passage = passages[0] if passages else None
    
    # Sprawdź czy to poprawny pasaż
    has_correct_passage = top_1_passage and top_1_passage.id == correct_passage_id
    
    if not has_correct_passage:
        missing_context_cases.append({
            'question_num': i + 1,
            'question': question,
            'correct_passage_id': correct_passage_id,
            'correct_answers': correct_answers,
            'retrieved_passage_id': top_1_passage.id if top_1_passage else None,
            'retrieved_context': top_1_passage.context if top_1_passage else None,
        })

print(f"\n{'='*80}")
print(f"Znaleziono {len(missing_context_cases)} pytań bez poprawnego kontekstu")
print(f"To stanowi {len(missing_context_cases)/len(polqa_dataset)*100:.1f}% wszystkich pytań")
print(f"{'='*80}")


Znaleziono 39 pytań bez poprawnego kontekstu
To stanowi 39.0% wszystkich pytań


## Wyświetl Szczegółowe Przypadki

In [6]:
# Wyświetl wszystkie przypadki
for i, case in enumerate(missing_context_cases, 1):
    print(f"\n{'='*80}")
    print(f"PRZYPADEK {i}/{len(missing_context_cases)} - Pytanie #{case['question_num']}")
    print(f"{'='*80}")
    
    print(f"\n📝 PYTANIE:")
    print(f"{case['question']}")
    
    print(f"\n✅ POPRAWNA ODPOWIEDŹ:")
    if isinstance(case['correct_answers'], list):
        for ans in case['correct_answers']:
            print(f"  - {ans}")
    else:
        print(f"  {case['correct_answers']}")
    
    print(f"\n🎯 OCZEKIWANY PASSAGE ID: {case['correct_passage_id']}")
    print(f"❌ ZWRÓCONY PASSAGE ID: {case['retrieved_passage_id']}")
    
    print(f"\n📄 ZWRÓCONY KONTEKST (błędny):")
    if case['retrieved_context']:
        # Wyświetl pierwsze 500 znaków kontekstu
        context_preview = case['retrieved_context'][:500]
        print(f"{context_preview}...")
    else:
        print("BRAK KONTEKSTU")
    
    print(f"\n{'-'*80}")


PRZYPADEK 1/39 - Pytanie #5

📝 PYTANIE:
Która polska drużyna nazywana jest Białą Gwiazdą?

✅ POPRAWNA ODPOWIEDŹ:
  - Wisła Kraków

🎯 OCZEKIWANY PASSAGE ID: 2513bc1d9e8e4b7304d50acbe41a13b837979bfec4cf3ef9b2e6dcdb777568df
❌ ZWRÓCONY PASSAGE ID: 628dcb389674aca8a468636c51ecdd9c0b6217345bc0e4b8088d15171d5cfba7

📄 ZWRÓCONY KONTEKST (błędny):
Strój zakonny. Strój zakonny składa się z czarnego habitu z białym kołnierzykiem oraz czarnego welonu z białą wypustką. Habit przepasany jest czarnym sznurem, na piersi zawieszony jest krzyż. W okresie letnim możliwy jest strój w kolorze szarym....

--------------------------------------------------------------------------------

PRZYPADEK 2/39 - Pytanie #7

📝 PYTANIE:
Którego bohatera narodowego przedstawia pomnik przy dolnej bramie wiodącej na Wawel?

✅ POPRAWNA ODPOWIEDŹ:
  - Tadeusza Kościuszkę
  - Kościuszkę

🎯 OCZEKIWANY PASSAGE ID: 6d43250faa96d4b5bc259f2dc4a0b9661316f62c878b678f0d237e31dc8b04b3
❌ ZWRÓCONY PASSAGE ID: f44df4e9cab93f3cd1d536d70a

## Statystyki

In [7]:
print(f"\n{'='*80}")
print("PODSUMOWANIE STATYSTYK")
print(f"{'='*80}")

total_questions = len(polqa_dataset)
missing_count = len(missing_context_cases)
found_count = total_questions - missing_count

print(f"\nCałkowita liczba pytań: {total_questions}")
print(f"Pytania z poprawnym kontekstem: {found_count} ({found_count/total_questions*100:.1f}%)")
print(f"Pytania bez poprawnego kontekstu: {missing_count} ({missing_count/total_questions*100:.1f}%)")

print(f"\n{'='*80}")
print("WNIOSKI:")
print(f"{'='*80}")
print(f"""\nDla konfiguracji basic_index z top_n=1:
- Retriever nie znalazł poprawnego kontekstu w {missing_count} przypadkach
- To oznacza, że model musiał generować odpowiedzi na podstawie błędnego kontekstu
- W tych przypadkach model może:
  1. Zahallucynować odpowiedź (FP - False Positive)
  2. Powiedzieć "Nie wiem" (TN - True Negative - poprawna odmowa)
  3. Wyciągnąć odpowiedź z błędnego kontekstu (FP - ale wygląda wiarygodnie)
""")


PODSUMOWANIE STATYSTYK

Całkowita liczba pytań: 100
Pytania z poprawnym kontekstem: 61 (61.0%)
Pytania bez poprawnego kontekstu: 39 (39.0%)

WNIOSKI:

Dla konfiguracji basic_index z top_n=1:
- Retriever nie znalazł poprawnego kontekstu w 39 przypadkach
- To oznacza, że model musiał generować odpowiedzi na podstawie błędnego kontekstu
- W tych przypadkach model może:
  1. Zahallucynować odpowiedź (FP - False Positive)
  2. Powiedzieć "Nie wiem" (TN - True Negative - poprawna odmowa)
  3. Wyciągnąć odpowiedź z błędnego kontekstu (FP - ale wygląda wiarygodnie)



## Eksport do CSV dla Dalszej Analizy

In [8]:
import csv
import os

# Stwórz katalog jeśli nie istnieje
os.makedirs("../../output/context_analysis", exist_ok=True)

# Zapisz do CSV
output_file = "../../output/context_analysis/missing_passages_basic_index_polqa.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    writer.writerow([
        "question_num",
        "question",
        "correct_answers",
        "correct_passage_id",
        "retrieved_passage_id",
        "retrieved_context"
    ])
    
    for case in missing_context_cases:
        correct_ans = " | ".join(case['correct_answers']) if isinstance(case['correct_answers'], list) else case['correct_answers']
        writer.writerow([
            case['question_num'],
            case['question'],
            correct_ans,
            case['correct_passage_id'],
            case['retrieved_passage_id'],
            case['retrieved_context']
        ])

print(f"\n✅ Zapisano {len(missing_context_cases)} przypadków do: {output_file}")


✅ Zapisano 39 przypadków do: ../../output/context_analysis/missing_passages_basic_index_polqa.csv


## Analiza Odpowiedzi Modelu na Błędny Kontekst

Teraz sprawdźmy jakie odpowiedzi wygenerował model dla tych pytań z błędnym kontekstem.

In [9]:
from generators.instruction_generator import InstructionGenerator

# Załaduj generator
model_path = "../../models/PLLuM-8B-instruct-q4"
generator = InstructionGenerator(model_path, cache)

print(f"Załadowano generator: {model_path}")
print("\nGenerowanie odpowiedzi dla pytań bez poprawnego kontekstu...")
print("(To może chwilę potrwać - używamy cache jeśli dostępny)\n")

Załadowano generator: ../../models/PLLuM-8B-instruct-q4

Generowanie odpowiedzi dla pytań bez poprawnego kontekstu...
(To może chwilę potrwać - używamy cache jeśli dostępny)



In [10]:
from tqdm import tqdm

# Dla każdego przypadku wygeneruj odpowiedź
for case in tqdm(missing_context_cases, desc="Generowanie odpowiedzi"):
    question = case['question']
    
    # Pobierz retriever result ponownie
    retriever_result = retriever.get_relevant_passages(question)
    passages = [passage for (passage, _) in retriever_result.passages]
    top_1_passage = passages[:1]
    
    # Wygeneruj odpowiedź (użyje cache jeśli dostępny)
    answer = generator.generate_answer(question, top_1_passage)
    
    # Sprawdź czy to odmowa odpowiedzi
    refusal_keywords = ["nie udało", "brak", "nie znaleziono", "nie wiem", "nie ma", "nie mogę"]
    is_refusal = any(kw in answer.lower() for kw in refusal_keywords)
    
    case['generated_answer'] = answer
    case['is_refusal'] = is_refusal

print("\n✅ Wygenerowano wszystkie odpowiedzi!")

Generowanie odpowiedzi: 100%|██████████| 39/39 [00:00<00:00, 707.30it/s]


✅ Wygenerowano wszystkie odpowiedzi!


## Klasyfikacja Odpowiedzi

In [11]:
# Policz statystyki
total_missing = len(missing_context_cases)
refusals = sum(1 for case in missing_context_cases if case['is_refusal'])
hallucinations = total_missing - refusals

print(f"\n{'='*80}")
print("KLASYFIKACJA ODPOWIEDZI DLA PYTAŃ BEZ POPRAWNEGO KONTEKSTU")
print(f"{'='*80}")

print(f"\nCałkowita liczba pytań bez poprawnego kontekstu: {total_missing}")
print(f"\n🟢 Odpowiedzi typu 'Nie wiem' (TN - True Negative): {refusals} ({refusals/total_missing*100:.1f}%)")
print(f"   → Model poprawnie odmówił odpowiedzi gdy nie miał dobrego kontekstu")

print(f"\n🔴 Odpowiedzi z błędnego kontekstu (FP - False Positive): {hallucinations} ({hallucinations/total_missing*100:.1f}%)")
print(f"   → Model wygenerował odpowiedź mimo braku poprawnego kontekstu (hallucynacja)")

print(f"\n{'='*80}")
print("WNIOSKI:")
print(f"{'='*80}")

if refusals > hallucinations:
    print(f"\n✅ Model jest KONSERWATYWNY - częściej odmawia niż hallucynuje")
    print(f"   Hallucination Rate: {hallucinations/total_missing*100:.1f}%")
    print(f"   Appropriate Refusal Rate: {refusals/total_missing*100:.1f}%")
else:
    print(f"\n⚠️ Model jest AGRESYWNY - częściej hallucynuje niż odmawia")
    print(f"   Hallucination Rate: {hallucinations/total_missing*100:.1f}%")
    print(f"   Appropriate Refusal Rate: {refusals/total_missing*100:.1f}%")


KLASYFIKACJA ODPOWIEDZI DLA PYTAŃ BEZ POPRAWNEGO KONTEKSTU

Całkowita liczba pytań bez poprawnego kontekstu: 39

🟢 Odpowiedzi typu 'Nie wiem' (TN - True Negative): 18 (46.2%)
   → Model poprawnie odmówił odpowiedzi gdy nie miał dobrego kontekstu

🔴 Odpowiedzi z błędnego kontekstu (FP - False Positive): 21 (53.8%)
   → Model wygenerował odpowiedź mimo braku poprawnego kontekstu (hallucynacja)

WNIOSKI:

⚠️ Model jest AGRESYWNY - częściej hallucynuje niż odmawia
   Hallucination Rate: 53.8%
   Appropriate Refusal Rate: 46.2%


## Przykłady Odpowiedzi

In [12]:
# Pokaż przykłady odmów
refusal_examples = [case for case in missing_context_cases if case['is_refusal']][:3]

if refusal_examples:
    print(f"\n{'='*80}")
    print(f"PRZYKŁADY POPRAWNYCH ODMÓW (TN - True Negative)")
    print(f"{'='*80}")
    
    for i, case in enumerate(refusal_examples, 1):
        print(f"\n--- PRZYKŁAD {i} ---")
        print(f"Pytanie: {case['question']}")
        print(f"Odpowiedź modelu: {case['generated_answer']}")
        print(f"✅ Model poprawnie odmówił odpowiedzi")

# Pokaż przykłady hallucynacji
hallucination_examples = [case for case in missing_context_cases if not case['is_refusal']][:3]

if hallucination_examples:
    print(f"\n{'='*80}")
    print(f"PRZYKŁADY HALLUCYNACJI (FP - False Positive)")
    print(f"{'='*80}")
    
    for i, case in enumerate(hallucination_examples, 1):
        print(f"\n--- PRZYKŁAD {i} ---")
        print(f"Pytanie: {case['question']}")
        print(f"Poprawna odpowiedź: {case['correct_answers']}")
        print(f"Odpowiedź modelu: {case['generated_answer']}")
        print(f"❌ Model zahallucynował odpowiedź z błędnego kontekstu")


PRZYKŁADY POPRAWNYCH ODMÓW (TN - True Negative)

--- PRZYKŁAD 1 ---
Pytanie: Która polska drużyna nazywana jest Białą Gwiazdą?
Odpowiedź modelu: nie udało mi się odnaleźć odpowiedzi na pytanie
✅ Model poprawnie odmówił odpowiedzi

--- PRZYKŁAD 2 ---
Pytanie: Które miasto jest stolicą Norwegii?
Odpowiedź modelu: nie udało mi się odnaleźć odpowiedzi na pytanie
✅ Model poprawnie odmówił odpowiedzi

--- PRZYKŁAD 3 ---
Pytanie: Przewodniczącym której partii był do momentu objęcia urzędu prezydenta Aleksander Kwaśniewski?
Odpowiedź modelu: nie udało mi się odnaleźć odpowiedzi na pytanie
✅ Model poprawnie odmówił odpowiedzi

PRZYKŁADY HALLUCYNACJI (FP - False Positive)

--- PRZYKŁAD 1 ---
Pytanie: Którego bohatera narodowego przedstawia pomnik przy dolnej bramie wiodącej na Wawel?
Poprawna odpowiedź: ['Tadeusza Kościuszkę', 'Kościuszkę']
Odpowiedź modelu: tadeusza kościuszki
❌ Model zahallucynował odpowiedź z błędnego kontekstu

--- PRZYKŁAD 2 ---
Pytanie: Jaką nazwę nosił główny kierunek śr

## Eksport Pełnej Analizy

In [13]:
# Zapisz pełną analizę z odpowiedziami
output_file_full = "../../output/context_analysis/missing_passages_with_answers_basic_index_polqa.csv"

with open(output_file_full, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    writer.writerow([
        "question_num",
        "question",
        "correct_answers",
        "correct_passage_id",
        "retrieved_passage_id",
        "retrieved_context",
        "generated_answer",
        "is_refusal",
        "classification"
    ])
    
    for case in missing_context_cases:
        correct_ans = " | ".join(case['correct_answers']) if isinstance(case['correct_answers'], list) else case['correct_answers']
        classification = "TN (Appropriate Refusal)" if case['is_refusal'] else "FP (Hallucination)"
        
        writer.writerow([
            case['question_num'],
            case['question'],
            correct_ans,
            case['correct_passage_id'],
            case['retrieved_passage_id'],
            case['retrieved_context'],
            case['generated_answer'],
            case['is_refusal'],
            classification
        ])

print(f"\n✅ Zapisano pełną analizę do: {output_file_full}")
print(f"\nPlik zawiera:")
print(f"  - {len(missing_context_cases)} pytań bez poprawnego kontekstu")
print(f"  - {refusals} odpowiedzi typu TN (poprawne odmowy)")
print(f"  - {hallucinations} odpowiedzi typu FP (hallucynacje)")


✅ Zapisano pełną analizę do: ../../output/context_analysis/missing_passages_with_answers_basic_index_polqa.csv

Plik zawiera:
  - 39 pytań bez poprawnego kontekstu
  - 18 odpowiedzi typu TN (poprawne odmowy)
  - 21 odpowiedzi typu FP (hallucynacje)
